In [1]:
!unzip sailu_lora.zip

Archive:  sailu_lora.zip
   creating: sailu_lora/
  inflating: sailu_lora/adapter_config.json  
  inflating: sailu_lora/adapter_model.safetensors  
  inflating: sailu_lora/README.md    
  inflating: sailu_lora/special_tokens_map.json  
  inflating: sailu_lora/spiece.model  
  inflating: sailu_lora/tokenizer.json  
  inflating: sailu_lora/tokenizer_config.json  
Archive:  t5-medical-lora-krish.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of t5-medical-lora-krish.zip or
        t5-medical-lora-krish.zip.zip, and cannot find t5-medical-lora-krish.zip.ZIP, period.


In [2]:
!unzip t5-medical-lora-krish.zip

Archive:  t5-medical-lora-krish.zip
   creating: t5-medical-lora-krish/
  inflating: t5-medical-lora-krish/adapter_config.json  
  inflating: t5-medical-lora-krish/adapter_model.safetensors  
   creating: t5-medical-lora-krish/checkpoint-354/
  inflating: t5-medical-lora-krish/checkpoint-354/adapter_config.json  
  inflating: t5-medical-lora-krish/checkpoint-354/adapter_model.safetensors  
  inflating: t5-medical-lora-krish/checkpoint-354/optimizer.pt  
  inflating: t5-medical-lora-krish/checkpoint-354/README.md  
  inflating: t5-medical-lora-krish/checkpoint-354/rng_state.pth  
  inflating: t5-medical-lora-krish/checkpoint-354/scaler.pt  
  inflating: t5-medical-lora-krish/checkpoint-354/scheduler.pt  
  inflating: t5-medical-lora-krish/checkpoint-354/special_tokens_map.json  
  inflating: t5-medical-lora-krish/checkpoint-354/tokenizer.json  
  inflating: t5-medical-lora-krish/checkpoint-354/tokenizer_config.json  
  inflating: t5-medical-lora-krish/checkpoint-354/trainer_state.json  

In [3]:
!pip install evaluate sacrebleu sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 48.7 MB/s eta 0:00:00


In [4]:
import torch
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import PeftModel
import evaluate

In [5]:
df = pd.read_csv("/content/test_data.csv")

inputs = ["simplify: " + x for x in df["complex"].astype(str).tolist()]
references = df["simple"].astype(str).tolist()

In [6]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Vanilla T5-small
vanilla = T5ForConditionalGeneration.from_pretrained(
    "t5-small"
).to(device).eval()

# 2. mrm8488 fine-tuned model
mrm = T5ForConditionalGeneration.from_pretrained(
    "mrm8488/t5-small-finetuned-text-simplification"
).to(device).eval()

# 3. Google Flan T5-base
flan_t5_base = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-base"
).to(device).eval()

# 4. LoRA on vanilla Google Flan T5-base
base_flan_vanilla = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
lora_vanilla = PeftModel.from_pretrained(
    base_flan_vanilla,
    "/content/sailu_lora"
).to(device).eval()

# 5. LoRA on mrm8488 fine-tuned Google Flan T5-base
base_flan_mrm = T5ForConditionalGeneration.from_pretrained(
    "mrm8488/t5-small-finetuned-text-simplification"
)
lora_mrm = PeftModel.from_pretrained(
    base_flan_mrm,
    "/content/t5-medical-lora-krish"
).to(device).eval()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
GEN_KWARGS = dict(
    max_length=128,
    num_beams=4,
    do_sample=False
)

def generate_all(model, texts, batch_size=8):
    outputs = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        enc = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(device)

        with torch.no_grad():
            gen = model.generate(**enc, **GEN_KWARGS)

        outputs.extend(
            tokenizer.batch_decode(gen, skip_special_tokens=True)
        )

    return outputs

In [10]:
out_vanilla = generate_all(vanilla, inputs)
out_mrm = generate_all(mrm, inputs)
out_flan_t5_base = generate_all(flan_t5_base, inputs)
out_lora_vanilla = generate_all(lora_vanilla, inputs)
out_lora_mrm = generate_all(lora_mrm, inputs)

In [11]:
sari = evaluate.load("sari")

def compute_sari(preds):
    return sari.compute(
        sources=inputs,
        predictions=preds,
        references=[[r] for r in references]
    )["sari"]

sari_scores = {
    "Vanilla T5": compute_sari(out_vanilla),
    "MRM8488": compute_sari(out_mrm),
    "Flan T5 Base": compute_sari(out_flan_t5_base),
    "LoRA T5": compute_sari(out_lora_vanilla),
    "LoRA MRM8488": compute_sari(out_lora_mrm),
}

sari_scores

{'Vanilla T5': 37.139214544328865,
 'MRM8488': 38.551458497694775,
 'Flan T5 Base': 37.57012831302298,
 'LoRA T5': 37.62872407774347,
 'LoRA MRM8488': 39.7967768473398}

In [12]:
bleu = evaluate.load("bleu")

def compute_bleu(preds):
    return bleu.compute(
        predictions=preds,
        references=[[r] for r in references]
    )["bleu"]

bleu_scores = {
    "Vanilla T5": compute_bleu(out_vanilla),
    "MRM8488": compute_bleu(out_mrm),
    "Flan T5 Base": compute_bleu(out_flan_t5_base),
    "LoRA T5": compute_bleu(out_lora_vanilla),
    "LoRA MRM8488": compute_bleu(out_lora_mrm),
}

bleu_scores

{'Vanilla T5': 0.03726169881709895,
 'MRM8488': 0.01836742142776142,
 'Flan T5 Base': 0.009147976857174066,
 'LoRA T5': 0.010877716103631835,
 'LoRA MRM8488': 0.06989384946133559}

In [17]:
idx = 3  # change if you want a different example

print("INPUT:")
print(df.loc[idx, "complex"])

print("\nREFERENCE:")
print(df.loc[idx, "simple"])

print("\nVANILLA T5:")
print(out_vanilla[idx])

print("\nMRM8488:")
print(out_mrm[idx])

print("\nFlan T5 Base:")
print(out_flan_t5_base[idx])

print("\nLoRA on T5-Flan:")
print(out_lora_vanilla[idx])

print("\nLoRA on MRM8488:")
print(out_lora_mrm[idx])

INPUT:
We included two RCTs (involving 154 women) with a low risk of bias. It was not possible to pool the available studies. The two studies do not consistently report outcomes defined in the review. However, no significant differences were observed between the two groups (surgical versus non-surgical repair) in incidence of pain and wound complications, self-evaluated measures of pain at hospital discharge and postpartum and re-initiation of sexual activity. Differences in the use of analgesia varied between the studies, being high in the sutured group in one study. The other trial showed differences in wound closure and poor wound approximation in the non-suturing group, but noted incidentally also that more women were breastfeeding in this group. There is limited evidence available from RCTs to guide the choice between surgical or non-surgical repair of first- or second-degree perineal tears sustained during childbirth. Two studies find no difference between the two types of manage

In [14]:
results_df = pd.DataFrame({
    "Model": list(sari_scores.keys()),
    "SARI": list(sari_scores.values()),
    "BLEU": [bleu_scores[k] for k in sari_scores.keys()]
})

results_df

,Model,SARI,BLEU
0,Vanilla T5,37.139215,0.037262
1,MRM8488,38.551458,0.018367
2,Flan T5 Base,37.570128,0.009148
3,LoRA T5,37.628724,0.010878
4,LoRA MRM8488,39.796777,0.069894
